In [13]:
from glob import glob
import pandas as pd
lst_file = glob("./records/交易明細*")
print(len(lst_file),'筆', lst_file[0])

17 筆 ./records\交易明細20211006.xlsx


In [8]:
df_all = pd.DataFrame(columns=['交易別', '代碼', '商品名稱', '成交股數', '成交單價', '成交價金', '手續費', '交易稅', '融資自備款', '融資金額', '融券擔保品', '融券保證金', '融券費', '淨收付', '交易日期'])

for file in lst_file:
    df = pd.read_excel(file, engine='openpyxl', dtype={'代碼':str})
    for idx, row in df.iterrows():
        if row['成交股數'] == 1000:
            df_all = df_all.append(row)
        else:   
            n = row['成交股數'] // 1000
            row['成交股數'] //= n
            row['成交價金'] //= n
            row['手續費'] //= n
            row['交易稅'] //= n
            row['淨收付'] //= n
            for i in range(n):
                df_all = df_all.append(row)
                
from datetime import datetime
df_all['交易日期'] = df_all['交易日期'].apply(lambda x : datetime.strptime(str(x), "%Y%m%d").date())
df_all.sort_values(['代碼', '交易日期'], inplace=True)
df_all.drop(['融資自備款','融資金額','融券擔保品','融券保證金', '融券費'], axis=1, inplace=True)
df_all.head(2)

,交易別,代碼,商品名稱,成交股數,成交單價,成交價金,手續費,交易稅,淨收付,交易日期
0,現買,00881,國泰台灣5G+,1000,16.6,16600,23,0,-16624,2021-10-06
0,現買,00881,國泰台灣5G+,1000,16.6,16600,23,0,-16624,2021-10-06


# 已賣出的股票，從清單上去除掉

In [9]:
df_buy = df_all[df_all['交易別']=='現買']
df_buy = df_buy.reset_index(drop=True)
df_buy.reset_index(inplace=True)
df_buy.head(1)

df_sold = df_all[df_all['交易別']=='現賣']
df_sold.reset_index(inplace=True, drop=True)
df_sold = df_sold.copy()
df_sold.rename(columns={'成交單價':'賣出單價', '淨收付':'賣出總價'}, inplace=True)
df_sold['交易月份'] = df_sold['交易日期'].apply(lambda x: x.month)
df_sold[['買入日期', '買入單價', '買入總價']] = 0

for idx, row in df_sold.iterrows():
    fst_row = df_buy[df_buy['代碼']== row['代碼']].iloc[0,:]
    df_sold.loc[idx, '買入日期'] = fst_row['交易日期']
    df_sold.loc[idx, '買入單價'] = fst_row['成交單價']
    df_sold.loc[idx, '買入總價'] = fst_row['淨收付']
    df_buy.drop(index=fst_row['index'], inplace=True)

# 尚未賣出的股票

In [10]:
df_remain = df_buy.copy()
df_remain.drop('index', axis=1, inplace=True)
path_remain = './remain.xlsx'
df_remain.to_excel(path_remain, index=False)
df_remain.head(2)

,交易別,代碼,商品名稱,成交股數,成交單價,成交價金,手續費,交易稅,淨收付,交易日期
0,現買,00881,國泰台灣5G+,1000,16.6,16600,23,0,-16624,2021-10-06
1,現買,00881,國泰台灣5G+,1000,16.6,16600,23,0,-16624,2021-10-06


# 已賣出的股票，計算報酬率

In [11]:
df_sold['賣出總價'] = df_sold['賣出總價'].astype(int)
df_sold['損益'] = df_sold['賣出總價'] + df_sold['買入總價']
df_sold['報酬率'] = round(df_sold['損益'] / df_sold['買入總價'] * (-100), 1)
df_sold.sort_values('交易日期', ascending=False, inplace=True)
path_return = './return.xlsx'
df_sold.to_excel(path_return, index=False)
df_sold.head(2)

,交易別,代碼,商品名稱,成交股數,賣出單價,成交價金,手續費,交易稅,賣出總價,交易日期,交易月份,買入日期,買入單價,買入總價,損益,報酬率
7,現賣,2603,長榮,1000,135.0,135000,192,405,134403,2021-12-15,12,2021-12-02,134.0,-134190,213,0.2
5,現賣,2603,長榮,1000,141.0,141000,200,423,140376,2021-12-10,12,2021-12-02,134.0,-134191,6185,4.6


# 計算每月賣出股票的報酬率

In [12]:
df_monthly_return = df_sold.groupby('交易月份').sum()
df_monthly_return['報酬率'] = round(df_monthly_return['損益'] / df_monthly_return['買入總價'] * (-100), 1)
df_monthly_return = df_monthly_return[['賣出總價', '買入總價', '損益', '報酬率']]
path_monthly_return = './monthly_return.xlsx'
df_monthly_return.to_excel(path_monthly_return)
df_monthly_return.head(2)

,賣出總價,買入總價,損益,報酬率
交易月份,,,,
10,264425,-263072,1353,0.5
11,583310,-574414,8896,1.5
